In [8]:
import sys, os
sys.path.append(os.pardir)
# 부모 디렉토리에서 import할 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
# mnist data load할 수 있는 함수 import
from PIL import Image
# python image processing library
# python 버전 3.x 에서는 pillow package install해서 사

In [9]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
# training data, test data
# flatten: 이미지를 1차원 배열로 읽음
# normalize: 0~1 실수로. 그렇지 않으면 0~255
image = x_train[0]
label = t_train[0]
# 첫번째 데이터
print(label)
print(image.shape)

5
(784,)


In [10]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    # image를 unsigned int로
    
image = image.reshape(28,28)
# 1차원 —> 2차원 (28x28)
print(image.shape)
img_show(image)

(28, 28)


In [11]:
import import_ipynb
from hwmyknn import MyKNeighborsClassifier

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score

In [146]:
# input feature를 hand-crafted feature로 가공한다.
# 총 784개의 input feature는 가로 28, 세로 28개의 픽셀로 구성되어 있으므로
# 가로세로 인접한 2개의 픽셀 씩 묶어, 총 네개의 픽셀을 하나의 픽셀로 압축한다.
# 압축을 위해서, 인접한 네 개의 픽셀의 수를 더하고, 4로 나눈다.
def compress_data(data): 
    rows = 7
    cols = 7
    compr = [[0 for j in range(cols)] for i in range(rows)]
    comp = np.array(compr)
    
    for k in range(7):
        for l in range(7):
            for i in range(4*k, 4*k+4):
                for j in range(4*l, 4*l+4):
                    comp[k][l] += data[i][j]
    for k in range(7):
        for l in range(7):
            comp[k][l] /= 4
    return comp

In [202]:
# 해당 블록의 코드는 input feature를 hand-crafted feature로 가공한다.
# 위의 compress_data function을 호출하여 data을 압축한다.
hand_crafted_train = [0 for i in range(len(x_train))]
np.array(hand_crafted_train)
for i in range(len(x_train)):
    hand_crafted_train[i] = x_train[i].reshape(28,28)

crafted_train = []
np.array(crafted_train)
for i in range(len(hand_crafted_train)):
    crafted_train.append(compress_data(hand_crafted_train[i]))
    
hand_crafted_test = [0 for i in range(len(x_test))]
np.array(hand_crafted_test)
for i in range(len(x_test)):
    hand_crafted_test[i] = x_test[i].reshape(28,28)
    
crafted_test = []
np.array(crafted_test)
for i in range(len(hand_crafted_test)):
    crafted_test.append(compress_data(hand_crafted_test[i]))

In [234]:
# 앞선 과정에서 만들어진 crafted feature를 flatten한다.
# 즉, 아래의 과정에서 사용할 수 있도록 2차원 배열에서 1차원 배열로 차수를 줄인다.
x_crafted_test = [0 for i in range(len(x_test))]
np.array(x_crafted_test)
x_crafted_train = [0 for i in range(len(x_train))]
np.array(x_crafted_train)

for i in range(len(crafted_test)):
    x_crafted_test[i] = crafted_test[i].flatten()
for i in range(len(crafted_train)):
    x_crafted_train[i] = crafted_train[i].flatten()

print(x_crafted_test[0].shape)

(49,)


In [19]:
train_size = 1000
train_sample = np.random.randint(0, t_train.shape[0], train_size)
# 0~10000에서 숫자 10000개 골라서 train set의 index로 사용
test_size = 100
test_sample = np.random.randint(0, t_test.shape[0], test_size) 
# 0~10000에서 숫자 100개 골라서 test set의 index로 사용

''' 
X와 y는 각각 data와 target이나, 
test용 자료와 train용 자료를 _test와 _train 이라는 변수명으로 구분한다. 
X = x
y = t (실제 값)
'''
X_test = []
X_train = []
y_test = []
y_train = []
'''
아래의 x_train[i]와 x_test[i]를 crafted_train[i]와 crafted_test[i]로 바꾸면,
KNN algorithm에서 hand-crafted feature를 사용할 수 있다.
'''
for i in train_sample:
    X_train.append(x_train[i])
    y_train.append(t_train[i])

for i in test_sample:
    X_test.append(x_test[i])
    y_test.append(t_test[i])
    
X_test = np.array(X_test)
X_train = np.array(X_train)
y_test = np.array(y_test)
y_train = np.array(y_train)

label_name = ['0','1','2','3','4','5','6','7','8','9']

In [20]:
import importlib
import hwmyknn
importlib.reload(hwmyknn)
from hwmyknn import MyKNeighborsClassifier

importing Jupyter notebook from hwmyknn.ipynb


In [31]:
'''Instantiate learning model (k = 13)'''
K = 15
# hwmyknn 모듈의 MyKNeighborsClassifier class를 이용해 my_classifier라는 이름의 classifier를 초기화
my_classifier = MyKNeighborsClassifier(X_test, X_train, y_test, y_train, K) 

'Test 결과를 예측하기'''
my_pred = [] # majority vote를 이용한 test 결과를 저장하는 배열
my_weighted_pred = [] # weighted majority vote를 이용한 test 결과를 저장하는 배열

for i in range(len(X_test)):
    # KNeighbors = 해당 X_test로부터 K개의 neighbors가 가진 label들
    #print(len(X_test_c)) 100
    #print(len(X_train_c)) 1000
    KNeighbors = my_classifier.get_KNeighbors(X_test[i], X_train, y_train)
    KNeighbors_with_dist = my_classifier.get_KNeighbors_with_dist(X_test[i], X_train, y_train)
    
    print(KNeighbors) # 해당 X_test로부터 K개의 neighbors가 가진 label 출력
    
    my_pred.append(my_classifier.majorityVote(KNeighbors)) # 리스트에 추가
    my_weighted_pred.append(my_classifier.weightedMajorityVote(KNeighbors_with_dist))
    

<string>:22: RuntimeWarning: overflow encountered in ubyte_scalars


[1, 1, 1, 2, 1, 1, 1, 9, 1, 1, 1, 1, 1, 1, 1]
[7, 7, 9, 7, 7, 9, 9, 7, 7, 1, 7, 7, 1, 1, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 5, 6, 0, 0, 0]
[3, 1, 3, 0, 1, 5, 4, 8, 1, 3, 8, 1, 3, 4, 8]
[0, 0, 0, 0, 5, 0, 8, 0, 0, 8, 0, 8, 8, 5, 0]
[7, 9, 7, 9, 4, 4, 9, 4, 7, 7, 9, 9, 9, 9, 4]
[7, 8, 5, 1, 5, 1, 1, 5, 5, 9, 5, 3, 1, 1, 8]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 7, 9, 8, 9, 8, 7, 2, 9, 4, 9, 9, 1, 0, 3]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[8, 8, 9, 8, 3, 9, 9, 9, 8, 9, 9, 9, 9, 7, 9]
[0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[8, 8, 6, 8, 8, 0, 8, 8, 6, 2, 2, 8, 3, 5, 5]
[0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 2, 6, 0, 0, 0]
[9, 7, 9, 4, 9, 7, 9, 7, 9, 9, 9, 7, 9, 9, 9]
[5, 0, 6, 0, 5, 0, 6, 0, 0, 5, 5, 0, 0, 0, 3]
[0, 0, 0, 0, 0, 5, 2, 0, 0, 4, 2, 1, 8, 6, 6]
[0, 2, 0, 8, 8, 2, 1, 1, 1, 2, 2, 2, 5, 1, 2]
[8, 6, 6, 8, 8, 8, 8, 8, 6, 8, 0, 0, 6, 0, 8]
[3, 3, 0, 2, 3, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1]
[8, 0, 0, 9, 0, 0, 0, 0, 0, 8, 0, 5, 4, 8, 8]
[0, 1, 5, 5, 5, 1, 0, 5, 1, 5, 0, 

In [32]:
print('majority vote', my_pred) # majority vote로 계산된, X_test(10개)의 예측 label
print('weighted majority vote', my_weighted_pred) # weighted majority vote로 계산된, X_test(10개)의 예측 label
print('answer', y_test)

majority vote [1, 7, 0, 1, 0, 9, 1, 1, 9, 1, 9, 0, 8, 0, 9, 0, 0, 2, 8, 1, 0, 5, 8, 1, 1, 6, 4, 5, 7, 2, 9, 1, 0, 0, 1, 8, 9, 9, 8, 0, 1, 7, 2, 1, 5, 9, 0, 6, 7, 9, 0, 1, 1, 0, 0, 1, 6, 9, 2, 7, 0, 6, 1, 0, 7, 9, 8, 9, 1, 0, 1, 7, 6, 7, 1, 1, 3, 6, 8, 3, 5, 1, 7, 1, 3, 7, 0, 8, 8, 3, 1, 8, 0, 1, 7, 9, 4, 8, 7, 1]
weighted majority vote [1, 7, 0, 3, 0, 9, 5, 1, 9, 1, 9, 0, 8, 0, 9, 0, 0, 2, 8, 1, 0, 5, 8, 1, 1, 6, 4, 5, 7, 2, 9, 1, 0, 0, 1, 8, 9, 9, 9, 0, 1, 7, 2, 1, 5, 9, 0, 6, 7, 9, 0, 1, 1, 0, 0, 8, 6, 9, 2, 7, 0, 6, 1, 0, 7, 9, 8, 9, 1, 0, 2, 7, 6, 7, 1, 1, 3, 6, 8, 3, 5, 1, 7, 1, 3, 7, 0, 8, 8, 3, 1, 8, 0, 1, 9, 9, 4, 8, 7, 1]
answer [1 7 0 3 0 9 5 1 7 1 8 0 8 0 9 5 0 2 8 3 8 5 8 4 8 6 4 5 9 2 9 1 0 3 8 8 9
 9 8 0 2 7 2 1 5 9 0 6 4 9 0 1 1 0 0 8 6 9 2 7 0 6 1 0 7 4 8 4 9 0 1 8 6 7
 2 5 3 6 8 9 5 1 7 1 3 7 0 8 8 3 2 8 0 1 9 9 4 8 7 0]


In [33]:
my_pred = np.array(my_pred)
my_weighted_pred = np.array(my_weighted_pred)
y_test = np.array(y_test)

In [34]:
for i in range(len(X_test)):
    print(test_sample[i], 'th Data: result ' , label_name[my_pred[i]] , ', label: ', label_name[y_test[i]])

accuracy = accuracy_score(y_test, my_pred)*100 # 실제 label인 y값과 나의 예측값을 비교해 정확도를 계산
print('Accuracy of our model is equal ' + str(round(accuracy, 2)) + ' %.')

7856 th Data: result  1 , label:  1
1705 th Data: result  7 , label:  7
7593 th Data: result  0 , label:  0
7501 th Data: result  1 , label:  3
2326 th Data: result  0 , label:  0
5856 th Data: result  9 , label:  9
4771 th Data: result  1 , label:  5
850 th Data: result  1 , label:  1
468 th Data: result  9 , label:  7
8418 th Data: result  1 , label:  1
5423 th Data: result  9 , label:  8
3803 th Data: result  0 , label:  0
3023 th Data: result  8 , label:  8
2972 th Data: result  0 , label:  0
3798 th Data: result  9 , label:  9
8578 th Data: result  0 , label:  5
9902 th Data: result  0 , label:  0
4247 th Data: result  2 , label:  2
6776 th Data: result  8 , label:  8
2222 th Data: result  1 , label:  3
4455 th Data: result  0 , label:  8
1940 th Data: result  5 , label:  5
7143 th Data: result  8 , label:  8
2901 th Data: result  1 , label:  4
2093 th Data: result  1 , label:  8
2006 th Data: result  6 , label:  6
95 th Data: result  4 , label:  4
2903 th Data: result  5 , label:

In [35]:
for i in range(len(X_test)):
    print(test_sample[i] , 'th Data: result ' , label_name[my_weighted_pred[i]] , ', label: ', label_name[y_test[i]])
    
accuracy = accuracy_score(y_test, my_weighted_pred)*100
print('Accuracy of our model with weighted majority vote is equal ' + str(round(accuracy, 2)) + ' %.')

7856 th Data: result  1 , label:  1
1705 th Data: result  7 , label:  7
7593 th Data: result  0 , label:  0
7501 th Data: result  3 , label:  3
2326 th Data: result  0 , label:  0
5856 th Data: result  9 , label:  9
4771 th Data: result  5 , label:  5
850 th Data: result  1 , label:  1
468 th Data: result  9 , label:  7
8418 th Data: result  1 , label:  1
5423 th Data: result  9 , label:  8
3803 th Data: result  0 , label:  0
3023 th Data: result  8 , label:  8
2972 th Data: result  0 , label:  0
3798 th Data: result  9 , label:  9
8578 th Data: result  0 , label:  5
9902 th Data: result  0 , label:  0
4247 th Data: result  2 , label:  2
6776 th Data: result  8 , label:  8
2222 th Data: result  1 , label:  3
4455 th Data: result  0 , label:  8
1940 th Data: result  5 , label:  5
7143 th Data: result  8 , label:  8
2901 th Data: result  1 , label:  4
2093 th Data: result  1 , label:  8
2006 th Data: result  6 , label:  6
95 th Data: result  4 , label:  4
2903 th Data: result  5 , label: